# Verification de l'implementation de la CL Echange_Externe_Radiatif

In [ ]:
from trustutils import run
import numpy as np

run.introduction("Elie Saikali & Yannick Gorsse")

run.TRUST_parameters()
run.reset()

def newton_T_paroi_VDF(eps, h, T_ext, flux, T0):
    Tb = T0
    it_max = 100
    tolerance = 1e-8
    for i in range(it_max):
        f = 5.67e-8 * eps * (Tb**4 - T_ext**4) + h * (Tb - T_ext) - flux
        f_p = 4 * 5.67e-8 * eps * Tb**3 + h
        Tb_new = Tb - f / f_p
        if abs(Tb_new - Tb) < tolerance:
            return Tb_new
        Tb = Tb_new
    raise RuntimeError("newton_T_paroi_VDF : newton did not converge !")

Text = 500
eps = [0, 0.7]
hs = [0, 100]

parameters = {
    "Text": Text,
    "dt" : 0.1,
}
dis = {"VDF" : ("", 11), "VEF" : ("trianguler_h dom", 11)}
fluxes = np.linspace(0, 1e5, 5)

nb_proc = 1
for d, (mesh, ny) in dis.items():
    for flux in fluxes:
        for e in eps:
            for h in hs:
                if e == 0 and h == 0: continue
                parameters["ny"] = ny
                parameters["e"] = e
                parameters["h"] = h
                parameters["flux"] = flux
                parameters["mesh"] = mesh
                parameters["dis"] = d
                tc = run.addCaseFromTemplate(f"jdd.data", f"{d}/e_{e}/h_{h}/flux_{flux:.1e}", {**parameters}, nb_proc)
                if nb_proc > 1: tc.partition()
run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

a = plot.Graph()
for d in dis:
    for flux in fluxes:
        for e in eps:
            for h in hs:
                if e == 0 and h == 0: continue
                a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/e_{e}/h_{h}/flux_{flux:.1e}/jdd.dt_ev", label=f"{d} - h={h} - flux_{flux:.1e}")

a.scale(yscale="log")

In [ ]:
for h in hs:
    for e in eps:
        if e == 0 and h == 0: continue
        a = plot.Graph(f"h = {h}, eps = {e}")
        for d in dis:
            temperatures = []
            exact = []
            for flux in fluxes:
                T = plot.loadText(f"{run.BUILD_DIRECTORY}/{d}/e_{e}/h_{h}/flux_{flux:.1e}/jdd_T.son")[-1][-1]
                temperatures.append(T)
                exact.append(newton_T_paroi_VDF(e, h, Text, flux, T))

            a.add(fluxes, temperatures, label=d)
        a.add(fluxes, exact, label="Analytique", marker="o")
        a.label("Flux (W/m2)", "Température")

for h in hs:
    for e in eps:
        if e == 0 and h == 0: continue
        a = plot.Graph(f"h = {h}, eps = {e}")
        for d in dis:
            a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/e_{e}/h_{h}/flux_{fluxes[-1]:.1e}/jdd_T_PROF.son", label=d, marker="o" if d =="VDF" else "-")